In [ ]:
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications.vgg16 import decode_predictions, preprocess_input, VGG16
import numpy as np
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
image_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
train_label = image_gen.flow_from_directory("../Datasets/train_images", target_size = (224,224), batch_size = 32, class_mode = "categorical")
test_label = image_gen.flow_from_directory("../Datasets/test_images", target_size = (224,224), batch_size = 1, class_mode = "categorical", 
                                          classes = train_label.class_indices)
valid_label = image_gen.flow_from_directory("../Datasets/valid_images", target_size = (224,224), batch_size = 32, class_mode = "categorical", 
                                           classes = train_label.class_indices)

In [ ]:
run = wandb.init(project = "My_first_project",
                 config = {
                     "learning_rate": 0.005,
                     "epochs": 5,
                     "loss_function": "categorical_crossentropy"
                 })
config = wandb.config

In [ ]:
tf.keras.backend.clear_session()

base_model = VGG16(include_top = False, weights = "imagenet", input_shape = (224,224,3), pooling = "max")
x = base_model.output
x = Dense(1024, activation = "relu")(x)
x = Dense(train_label.num_classes, activation = "softmax")(x)

model = Model(inputs = base_model.inputs, outputs = x)

for layer in base_model.layers :
    layer.trainable = False
    
model.compile(optimizer = tf.keras.optimizers.Adam(config.learning_rate), loss = config.loss_function, metrics = ["acc"])

In [ ]:
model.summary()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

wandb_callback = WandbCallback()
model.fit(train_label, epochs = config.epochs, steps_per_epoch = 100, validation_data = valid_label, callbacks = [wandb_callback])

In [ ]:
loss, accuracy = model.evaluate(test_label, verbose = 2)
print("Test Error Rate : ", round((1-accuracy)*100, 2))

wandb.log({"Test Error Rate":  round((1-accuracy)*100, 2)})
run.join()